# Poisson 1D

This notebook solves the linear problem:

$$ -u(x)'' = f(x) \text{ in } \Omega = [-1,1]$$ 

with

$$ f(x) = \frac{\pi^2}{4} \cos{\left(\frac{\pi x}{2}\right)} $$

and homogeneous Dirichlet boundary conditions using Hermite elements. The purpose is to find what the issues are while setting Dirichlet BCs and fix them. See [Manually rebuilding the mass matrix](#Manually-rebuilding-the-mass-matrix).

In [ ]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as pl
import FIAT
%matplotlib inline
u_exact = lambda x: np.cos(np.pi*x/2)

In [ ]:
def poisson1d(V):
    deg = V.ufl_element().degree()
    def boundary(x):
        return x[0] < -1 + DOLFIN_EPS or x[0] > 1.0 - DOLFIN_EPS

    # Need to project to work around lack of evaluate_dofs() for Hermite
    u0 = project(Constant(0.0), V)
    bc = DirichletBC(V, u0, boundary)

    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression("pi*pi*cos(pi*x[0]/2)/4", degree=deg)

    a = u.dx(0)*v.dx(0)*dx
    L = f*v*dx

    u = Function(V)
    solve(a == L, u, bc)
    return u

In [ ]:
#mesh = UnitIntervalMesh(2)
#MeshTransformation.translate(mesh, Point(1.0))
mesh = IntervalMesh(2, -1, 1)

V = FunctionSpace(mesh, "Lagrange", 3)
u = poisson1d(V)

In [ ]:
xx = np.linspace(-1, 1, 100)
pl.plot(xx, [u(x) for x in xx], label='$u_{l2}$')
pl.plot(xx, [u_exact(x) for x in xx], label='$u_{ex}$')
pl.ylim((0,1.1))
_ = pl.legend()

# Using Hermite elements

Naively applying Dirichlet boundary conditions fails because first derivatives are set to 0 when `DirichletBC` clears all the rows of dofs of nodes at the boundary, since this includes Hermite dofs associated to partial derivatives.

In [ ]:
V = FunctionSpace(mesh, "Hermite", 3)
u = poisson1d(V)

xx = np.linspace(-1, 1, 100)
pl.plot(xx, [u(x) for x in xx], label='$u_h$')
pl.plot(xx, [u_exact(x) for x in xx], label='$u_e$')
pl.ylim((0, 1.1))
_ = pl.legend()

## Manually rebuilding the stiffness matrix

The solution is to manually restore the relevant rows of the assembled matrix to their values before `DirichletBC.apply()` emptied them. The code has been factored out into [boundary.ipynb](boundary.ipynb).

Analogously, the way one applies Neumann boundary conditions has to be adapted in e.g. 4th order problems like the beam equation: in this case Neumann bcs are essential and are included in the function space by setting the relevant dofs to the desired values. See [Euler-Bernoulli.ipynb](Euler-Bernoulli.ipynb).

In [ ]:
import nbimporter
from boundary import apply_dirichlet_hermite #, plot_hermite_dofs
from utils import fnor, one_arg

def poisson1d_hermite(V):
    """
        V: FunctionSpace.
    """
    assert V.ufl_element().family() == 'Hermite', 'Please pay attention.'

    _left = V.mesh().coordinates().min()
    _right = V.mesh().coordinates().max()

    def left(x):
        return near(x[0], _left)
    def right(x):
        return near(x[0], _right)

    # Need to project to work around lack of evaluate_dofs() for Hermite
    u0 = project(Constant(0.0), V)
    bc = DirichletBC(V, u0, one_arg(fnor(left,right)))
    #plot_hermite_dofs(bc)

    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression("pi*pi*cos(pi*x[0]/2)/4",
                   degree=V.ufl_element().degree())

    a = u.dx(0)*v.dx(0)*dx
    F = f*v*dx

    A = assemble(a)
    b = assemble(F)

    apply_dirichlet_hermite(A, b, bc)

    u = Function(V)
    U = u.vector()
    solve(A, U, b)
    return u

In [ ]:
mesh = IntervalMesh(100, -1, 1)
V = FunctionSpace(mesh, "Hermite", 3)
u = poisson1d_hermite(V)

In [ ]:
xx = mesh.coordinates()
pl.plot(xx, [u(x) for x in xx], label='$u_h$')
pl.plot(xx, [u_exact(x) for x in xx], label='$u_e$')
pl.ylim((0, 1.1))
_ = pl.legend()